# BackTrader

In [14]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import numpy as np
import pandas as pd
import backtrader as bt

import tushare as ts
token = 'd9645bfd8516b93f1312d8ba696c83b606a78d966e71ec5c1e79bef4'
pro = ts.pro_api(token)

In [35]:
# 使用Tushare获取数据，要严格保持OHLC的格式
df = pro.daily(ts_code='600276.SH', start_date='20180101', end_date='20210731')
df = df[['trade_date', 'open', 'high', 'low', 'close','vol']]
df.trade_date = pd.to_datetime(df.trade_date)
# 索引必须是日期
df.index = df.trade_date
# 日期必须要升序
df.sort_index(inplace=True)
df

,trade_date,open,high,low,close,vol
trade_date,,,,,,
2018-01-02,2018-01-02,69.42,69.85,68.20,68.67,93925.46
2018-01-03,2018-01-03,68.69,70.00,68.60,68.83,124564.48
2018-01-04,2018-01-04,69.12,70.69,68.83,70.57,160499.09
2018-01-05,2018-01-05,71.55,71.78,69.88,70.32,133913.00
2018-01-08,2018-01-08,70.41,71.35,69.90,71.26,97948.32
...,...,...,...,...,...,...
2021-07-26,2021-07-26,61.30,61.30,58.59,59.22,464010.01
2021-07-27,2021-07-27,59.21,59.58,54.11,54.50,810946.44
2021-07-28,2021-07-28,53.97,56.27,53.60,55.91,523920.23


In [42]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' 
        日志函数：打印结果
        datas[0]：传入的数据，包含日期、OHLC等数据
        datas[0].datetime.date(0)：调用传入数据中的日期列
        '''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # dataclose变量：跟踪当前收盘价
        self.dataclose = self.datas[0].close

        # order变量：跟踪订单状态
        self.order = None

    def notify_order(self, order):
        '''订单状态通知(order.status)：提示成交状态'''
        if order.status in [order.Submitted, order.Accepted]:
            # 如果订单只是提交状态，那么啥也不提示
            return

        # 检查订单是否执行完毕
        # 注意：如果剩余现金不足，则会被拒绝！
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            
            # 记录订单执行的价格柱的编号（即长度）
            self.bar_executed = len(self)

        # 如果订单被取消/保证金不足/被拒绝
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # 如果没有查询到订单，则订单为None
        self.order = None

    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 检查一下是否有订单被挂起，如果有的话就先不下单
        if self.order:
            return
        
        # 检查一下目前是否持有头寸，如果没有就建仓
        # 策略：价格连续跌3天就买
        if not self.position:

            # 如果现价<昨日价格
            if self.dataclose[0] < self.dataclose[-1]:
                # 如果昨日价格<前日价格
                if self.dataclose[-1] < self.dataclose[-2]:
                    # 买买买！先记录一下买入价格（收盘价）
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    # 更新订单状态：buy()：开仓买入，买入价是下一个数据，即【开盘价】
                    self.order = self.buy()
        else:
             # 如果已经建仓，并持有头寸，则执行卖出指令
             # 如果当前价格柱已经距离执行订单过去5个，则卖出
            if len(self) >= (self.bar_executed + 5):
                # 卖!卖!卖!
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                # 更新订单状态：sell()：平仓卖出，卖出价是下一个数据，即【开盘价】
                self.order = self.sell()

# Create a cerebro entity
cerebro = bt.Cerebro()

 # Add a strategy
cerebro.addstrategy(TestStrategy)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=df)

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2018-01-02, Close, 68.67
2018-01-03, Close, 68.83
2018-01-04, Close, 70.57
2018-01-05, Close, 70.32
2018-01-08, Close, 71.26
2018-01-09, Close, 74.70
2018-01-10, Close, 74.90
2018-01-11, Close, 74.27
2018-01-12, Close, 74.61
2018-01-15, Close, 74.49
2018-01-16, Close, 75.10
2018-01-17, Close, 73.54
2018-01-18, Close, 72.80
2018-01-18, BUY CREATE, 72.80
2018-01-19, BUY EXECUTED, 72.88
2018-01-19, Close, 74.36
2018-01-22, Close, 76.39
2018-01-23, Close, 76.69
2018-01-24, Close, 78.07
2018-01-25, Close, 76.06
2018-01-26, Close, 77.15
2018-01-26, SELL CREATE, 77.15
2018-01-29, SELL EXECUTED, 77.51
2018-01-29, Close, 73.24
2018-01-30, Close, 74.10
2018-01-31, Close, 75.69
2018-02-01, Close, 74.23
2018-02-02, Close, 74.28
2018-02-05, Close, 73.37
2018-02-06, Close, 69.01
2018-02-06, BUY CREATE, 69.01
2018-02-07, BUY EXECUTED, 71.08
2018-02-07, Close, 67.69
2018-02-08, Close, 69.11
2018-02-09, Close, 67.56
2018-02-12, Close, 70.97
2018-02-13, Close, 71.23
2